<a href="https://colab.research.google.com/github/Devica2000/StanfordTech16LLM/blob/main/Devica_Verma_TECH16_LLM_Lecture2_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

# Task 1: Summarizing documents using stuff and map-reduce
Summarized the paper "Attention is All You Need" using stuff and map-reduce method. Later, compared the summaries using these two methods.

In [ ]:
# Get a PDF - Attention is All You Need
!wget https://arxiv.org/pdf/1706.03762.pdf

--2024-07-25 03:48:24--  https://arxiv.org/pdf/1706.03762.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/1706.03762 [following]
--2024-07-25 03:48:24--  http://arxiv.org/pdf/1706.03762
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘1706.03762.pdf’

1706.03762.pdf      100%[===================>]   2.11M  --.-KB/s    in 0.1s    

2024-07-25 03:48:24 (21.5 MB/s) - ‘1706.03762.pdf’ saved [2215244/2215244]



In [ ]:
!ls

1706.03762.pdf	sample_data


In [ ]:
!pip install -U langchain-community pypdf langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.7 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader # Update import statement

loader = PyPDFLoader("1706.03762.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(metadata={'source': '1706.03762.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\n

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4-turbo-preview", api_key=open_ai_key)
chain = load_summarize_chain(llm, chain_type="stuff")

#using 0:8 because the later pages would have irrelevant content
res = chain.invoke(pages[0:8])
stuff_summary = res["output_text"]

In [ ]:
print("Summary using stuff as the chain type:\n", stuff_summary)

Summary using stuff as the chain type:
 Google grants permission to reproduce tables and figures from the paper "Attention Is All You Need" by Ashish Vaswani and others, for journalistic or scholarly works with proper attribution. The paper introduces the Transformer model, which relies solely on attention mechanisms, eliminating the need for recurrent or convolutional neural networks. This architecture allows for more parallelization and significantly reduces training time. The Transformer model achieves state-of-the-art results on English-to-German and English-to-French translation tasks, outperforming existing models with a BLEU score of 28.4 and 41.8, respectively. The model also demonstrates strong performance on English constituency parsing. The Transformer's efficiency and effectiveness are attributed to its unique use of self-attention, which connects all positions in a sequence with a constant number of operations, facilitating learning of long-range dependencies. The paper de

In [ ]:
#using map-reduce for chain type

chain = load_summarize_chain(llm, chain_type="map_reduce")
res = chain.invoke(pages[0:8])
map_reduce_summary = res["output_text"]

In [ ]:
print("Summary using map-reduce as the chain type:\n",map_reduce_summary)

Summary using map-reduce as the chain type:
 Google has authorized the use of tables and figures from the "Attention Is All You Need" paper for journalistic or scholarly purposes with proper attribution. This paper, a collaboration between Google Brain, Google Research, and the University of Toronto, introduces the Transformer model, a novel neural network architecture that eschews traditional recurrent and convolutional layers in favor of an attention mechanism. This design allows for significant parallelization, improving efficiency and performance in machine translation tasks, evidenced by record-breaking BLEU scores on English-to-German and English-to-French translations with reduced training times. The Transformer model, which employs an encoder-decoder structure with multi-head self-attention and position-wise feed-forward networks, demonstrates a breakthrough in modeling global dependencies without sequential computation. It also incorporates innovations like Scaled Dot-Product 

### Comparing the stuff and map reduce summaries

In [ ]:
  prompt = f"""
  Summary using stuff as the chain type:
  {stuff_summary}

  Summary using map-reduce as the chain type:
  {map_reduce_summary}
  """

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  # response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are an AI that understands the text from a human given as input. Then, you compare the text and output the differences between the text in a table. You display the differences and similarities in separate tables. Be concise. Do not create content on your own."},
    {"role": "user", "content": prompt}
  ]
)

print(response.choices[0].message.content)

| Aspect                       | "Stuff as the chain" summary                                                                                                                                                                                           | "Map-reduce as the chain" summary                                                                                                                      |
|------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------|
| Source of Authorization       | Google grants permission to reproduce tables and figures from the paper "Attention Is All You Need" by Ashish Vaswani and others                              

## For practice: Build a Retrieval Augmented Generation (RAG) App
Followed the tutorial available on the official website of LangChain to learn how to build a simple RAG App.

In [ ]:
!pip install langchain langchain_community langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 11.2 MB/s eta 0:00:00

In [ ]:
# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
!pip install -qU langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key = open_ai_key)

#### Step 1: Indexing

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

print(len(docs[0].page_content))

43131


In [ ]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


#### Step 2: Splitting

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

66

In [ ]:
len(all_splits[0].page_content)

969

In [ ]:
all_splits[65].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 42190}

#### Step 3: Store

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(api_key = open_ai_key))

In [ ]:
vectorstore

#### Step 4: Retrieve

In [ ]:
#retriever returns relevant Documents given a string query
# converting the vectorstore into a retriever using as_retriever

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


#### Step 5: Generate

In [ ]:
!pip install -qU langchain-openai

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Don't try to make up an answer."
    "Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down a complicated task into smaller, more manageable steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance model performance on complex tasks by encouraging a step-by-step approach and exploring multiple reasoning possibilities. This allows for better planning and execution of tasks by transforming big tasks into simpler sub-tasks.


#### Another way

In [ ]:
# from langchain_core.prompts import PromptTemplate

# template = """Use the following pieces of context to answer the question at the end.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# Use three sentences maximum and keep the answer as concise as possible.
# Always say "thanks for asking!" at the end of the answer.

# {context}

# Question: {question}

# Helpful Answer:"""
# custom_rag_prompt = PromptTemplate.from_template(template)

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# #retriever | format_docs passes the question through the retriever
# #generating Document objects, and then to format_docs to generate strings
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | custom_rag_prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is the process of breaking down a complicated task into smaller, manageable steps or subgoals. Techniques like Chain of Thought (CoT) and Tree of Thoughts facilitate this by prompting the model to think step-by-step and explore multiple reasoning possibilities. This approach enhances model performance and provides insights into the model's thinking process. Thanks for asking!"

In [ ]:
#get the source
for document in response["context"]:
    print(document)
    print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS 

# Extra Credit: Querying a YouTube Video
The goal is to query a YouTube video i.e., ask anything related to the contents of the video, and the model will output an answer.
In this task, I have used a video from the MIT 6.S191: Introduction to
Deep Learning course. I picked up the second video lecture in this series which is about Deep Sequence Modeling. [Here](https://www.youtube.com/watch?v=dqoEU9Ac3ek&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=4) is link to the YouTube video.


In [ ]:
from langchain.document_loaders import YoutubeLoader
from langchain.indexes import VectorstoreIndexCreator


In [ ]:
!pip install langchain youtube-transcript-api openai chromadb pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
from langchain.document_loaders import YoutubeLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = open_ai_key
def extract_url(youtube_url_or_id):
    if "youtube.com" in youtube_url_or_id:
        if "?v=" in youtube_url_or_id:
            return youtube_url_or_id.split("?v=")[1]
        else:
            raise ValueError(f"Could not extract video ID from URL: {youtube_url_or_id}")
    else:
        return youtube_url_or_id

def load_and_split(youtube_url_or_id):
    video_id = extract_url(youtube_url_or_id)
    loader = YoutubeLoader.from_youtube_url(f"https://www.youtube.com/watch?v={video_id}", add_video_info=True)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    texts = text_splitter.split_documents(docs)

    return texts

# def summarize_content(texts):
#     llm = OpenAI(temperature=0)
#     chain = load_summarize_chain(llm, chain_type="map_reduce")
#     summary = chain.run(texts)
#     return summary

def setup_qa_chain():
    llm = ChatOpenAI(model="gpt-4o", api_key = open_ai_key)
    prompt_template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    Answer:"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)
    return chain

In [ ]:
!pip install tiktoken

In [ ]:
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain


# Example YouTube video ID with additional parameters
youtube_id = "dqoEU9Ac3ek&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=2"

# Load and split the content
texts = load_and_split(youtube_id)
print("Content loaded and split successfully.")

# Summarize the content
llm = ChatOpenAI(model="gpt-4o", api_key = open_ai_key)
chain = load_summarize_chain(llm, chain_type="stuff")
res = chain.invoke(texts)
summary = res["output_text"]

# summary = summarize_content(texts)
print("Content summarized successfully.")
print(f"Summary: {summary[:500]}...")  # Print first 500 characters of the summary

# Set up QA chain
qa_chain = setup_qa_chain()

# Hard-coded question
question = "What is the main topic of this video?"

# Create a Document object from the summary
summary_doc = Document(page_content=summary)

# Query the QA chain with the summarized content
response = qa_chain({"input_documents": [summary_doc], "question": question})
print(f"\nQuestion: {question}")
print(f"Answer: {response['output_text']}")

Content loaded and split successfully.
Content summarized successfully.
Summary: In this lecture, Ava introduces sequence modeling problems in the context of deep learning, building on foundational concepts from the first lecture. She explains how neural networks can handle sequential data, illustrated by the example of predicting the movement of a ball. Sequential data, including language, audio, and biological sequences, requires models that can process information over time. Ava introduces the concept of recurrent neural networks (RNNs) and their ability to maintain a sta...

Question: What is the main topic of this video?
Answer: The main topic of this video is sequence modeling problems in the context of deep learning, including the use of recurrent neural networks (RNNs), Long Short-Term Memory (LSTM) networks, and Transformer architectures for handling sequential data. The lecture covers the challenges of training RNNs, the concept of self-attention, and practical applications o

In [ ]:
from langchain.chains import RetrievalQA

def create_vectorstore(texts):
    embeddings = OpenAIEmbeddings(api_key = open_ai_key)
    vectorstore = Chroma.from_documents(texts, embeddings)
    return vectorstore

def setup_retrieval_qa_chain(vectorstore):
    llm = ChatOpenAI(model="gpt-4o", api_key=open_ai_key)
    prompt_template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    Answer:"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

# Example YouTube video ID with additional parameters
youtube_id = "dqoEU9Ac3ek&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=2"

# Load and split the content
texts = load_and_split(youtube_id)
print("Content loaded and split successfully.")

# Create vectorstore
vectorstore = create_vectorstore(texts)
print("Vectorstore created successfully.")

# Setting up Retrieval QA chain
qa_chain = setup_retrieval_qa_chain(vectorstore)

question = "How does Ava explain attention mechanism?"

# Query the QA chain
response = qa_chain({"query": question})
print(f"\nQuestion: {question}")
print(f"Answer: {response['result']}")

Content loaded and split successfully.
Vectorstore created successfully.

Question: How does Ava explain attention mechanism?
Answer: Ava explains the attention mechanism by describing it as a powerful and elegant concept that focuses on attending to and extracting the most important parts of an input. Specifically, she focuses on "Self attention," which involves attending to parts of the input itself. Ava uses the example of looking at an image of Iron Man to illustrate how attention can help identify important features without scanning each pixel individually. She also mentions that in the context of neural networks and Transformers, attention mechanisms use positional encodings to preserve information about the order and position of elements in a sequence. The attention mechanism involves computing three key components: the query, the key, and the value, which help determine the relevance of different parts of the input.
